In [0]:
!pip install mxnet-cu101
!pip install gluonnlp pandas tqdm
!pip install sentencepiece

In [0]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-sr5ew1of
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-sr5ew1of
  Created wheel for kobert: filename=kobert-0.1.1-cp36-none-any.whl size=12773 sha256=f45e0298ec377f15b43e29c26c6ef0fd01c9f9dfb98685debfb05f49682ae3aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-ba1sc0ws/wheels/a2/b0/41/435ee4e918f91918be41529283c5ff86cd010f02e7525aecf3
Successfully built kobert


In [0]:
import pandas as pd
import numpy as np
from mxnet.gluon import nn, rnn
from mxnet import gluon, autograd
import gluonnlp as nlp
from mxnet import nd 
import mxnet as mx
import time
import itertools
import random

from kobert.mxnet_kobert import get_mxnet_kobert_model
from kobert.utils import get_tokenizer

### Loading KoBERT

In [0]:
ctx = mx.gpu()

In [0]:
bert_base, vocab = get_mxnet_kobert_model(use_decoder=False, use_classifier=False, ctx=ctx)

using cached model
using cached model


In [0]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


### Loading Data

In [0]:
from google.colab import drive
drive.mount("/gdrive", force_remount = True)

Mounted at /gdrive


In [0]:
cd /gdrive/My \Drive

/gdrive/My Drive


In [0]:
dataset_train = nlp.data.TSVDataset("train_1.txt", num_discard_samples=1)  # header 제거
dataset_test = nlp.data.TSVDataset("test_1.txt", num_discard_samples=1)

In [0]:
class BERTDataset(mx.gluon.data.Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        sent_dataset = gluon.data.SimpleDataset([[
            i[sent_idx],
        ] for i in dataset])
        self.sentences = sent_dataset.transform(transform)
        self.labels = gluon.data.SimpleDataset(
            [np.array(np.int32(i[label_idx])) for i in dataset])

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [0]:
max_len = 256

# OOM issue 때문에 조절이 필요
# 128에 batch size 32로 하거나
# 256에 batch size 16으로 할 것
# 그 이상으로 긴 sequence들은 그냥 잘리게 됨

In [0]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [0]:
len(data_train)

22511

In [0]:
class BERTClassifier(nn.Block):
    def __init__(self,
                 bert,
                 num_classes=2,
                 dropout=None,
                 prefix=None,
                 params=None):
        super(BERTClassifier, self).__init__(prefix=prefix, params=params)
        self.bert = bert
        with self.name_scope():
            self.classifier = nn.HybridSequential(prefix=prefix)
            if dropout:
                self.classifier.add(nn.Dropout(rate=dropout))
            self.classifier.add(nn.Dense(units=num_classes))

    def forward(self, inputs, token_types, valid_length=None):
        _, pooler = self.bert(inputs, token_types, valid_length)
        return self.classifier(pooler)

In [0]:
model = BERTClassifier(bert_base, num_classes=2, dropout=0.1)
# 분류 레이어만 초기화 한다. 
model.classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
model.hybridize()

# softmax cross entropy loss for classification
loss_function = gluon.loss.SoftmaxCELoss()

metric = mx.metric.Accuracy()

In [0]:
batch_size = 16
lr = 5e-5

train_dataloader = mx.gluon.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = mx.gluon.data.DataLoader(data_test, batch_size=int(batch_size/2), num_workers=5)

In [0]:
trainer = gluon.Trainer(model.collect_params(), 'bertadam',
                        {'learning_rate': lr, 'epsilon': 1e-9, 'wd':0.01})

log_interval = 4
num_epochs = 5

In [0]:
# LayerNorm과 Bias에는 Weight Decay를 적용하지 않는다. 
for _, v in model.collect_params('.*beta|.*gamma|.*bias').items():
    v.wd_mult = 0.0
params = [
    p for p in model.collect_params().values() if p.grad_req != 'null'
]

In [0]:
def evaluate_accuracy(model, data_iter, ctx=ctx):
    acc = mx.metric.Accuracy()
    i = 0
    for i, (t,v,s, label) in enumerate(data_iter):
        token_ids = t.as_in_context(ctx)
        valid_length = v.as_in_context(ctx)
        segment_ids = s.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = model(token_ids, segment_ids, valid_length.astype('float32'))
        acc.update(preds=output, labels=label)
        if i > 1000:
            break
        i += 1
    return(acc.get()[1])

In [0]:
#learning rate warmup을 위한 준비 
accumulate = 4
step_size = batch_size * accumulate if accumulate else batch_size
num_train_examples = len(data_train)
num_train_steps = int(num_train_examples / step_size * num_epochs)
warmup_ratio = 0.1
num_warmup_steps = int(num_train_steps * warmup_ratio)
step_num = 0
all_model_params = model.collect_params()

In [0]:
# Set grad_req if gradient accumulation is required
if accumulate and accumulate > 1:
    for p in params:
        p.grad_req = 'add'

In [0]:
for epoch_id in range(num_epochs):
    metric.reset()
    step_loss = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(train_dataloader):
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            non_warmup_steps = step_num - num_warmup_steps
            offset = non_warmup_steps / (num_train_steps - num_warmup_steps)
            new_lr = lr - offset * lr
        trainer.set_learning_rate(new_lr)
        with mx.autograd.record():
            # load data to GPU
            token_ids = token_ids.as_in_context(ctx)
            valid_length = valid_length.as_in_context(ctx)
            segment_ids = segment_ids.as_in_context(ctx)
            label = label.as_in_context(ctx)

            # forward computation
            out = model(token_ids, segment_ids, valid_length.astype('float32'))
            ls = loss_function(out, label).mean()

        # backward computation
        ls.backward()
        if not accumulate or (batch_id + 1) % accumulate == 0:
          trainer.allreduce_grads()
          nlp.utils.clip_grad_global_norm(params, 1)
          trainer.update(accumulate if accumulate else 1)
          step_num += 1
          if accumulate and accumulate > 1:
              # set grad to zero for gradient accumulation
              all_model_params.zero_grad()

        step_loss += ls.asscalar()
        metric.update([label], [out])
        if (batch_id + 1) % (50) == 0:
            print('[Epoch {} Batch {}/{}] loss={:.4f}, lr={:.10f}, acc={:.3f}'
                         .format(epoch_id + 1, batch_id + 1, len(train_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, metric.get()[1]))
            step_loss = 0
    test_acc = evaluate_accuracy(model, test_dataloader, ctx)
    print('Test Acc : {}'.format(test_acc))

[Epoch 1 Batch 50/1407] loss=8.9114, lr=0.0000034286, acc=0.501
[Epoch 1 Batch 100/1407] loss=7.9868, lr=0.0000068571, acc=0.583
[Epoch 1 Batch 150/1407] loss=8.1951, lr=0.0000105714, acc=0.605
[Epoch 1 Batch 200/1407] loss=7.9683, lr=0.0000140000, acc=0.622
[Epoch 1 Batch 250/1407] loss=8.0039, lr=0.0000177143, acc=0.631
[Epoch 1 Batch 300/1407] loss=7.8539, lr=0.0000211429, acc=0.641
[Epoch 1 Batch 350/1407] loss=7.8489, lr=0.0000248571, acc=0.648
[Epoch 1 Batch 400/1407] loss=8.0319, lr=0.0000282857, acc=0.651
[Epoch 1 Batch 450/1407] loss=7.7213, lr=0.0000320000, acc=0.657
[Epoch 1 Batch 500/1407] loss=7.9853, lr=0.0000354286, acc=0.658
[Epoch 1 Batch 550/1407] loss=8.1245, lr=0.0000391429, acc=0.657
[Epoch 1 Batch 600/1407] loss=8.0175, lr=0.0000425714, acc=0.658
[Epoch 1 Batch 650/1407] loss=8.0751, lr=0.0000462857, acc=0.658
[Epoch 1 Batch 700/1407] loss=7.9791, lr=0.0000497143, acc=0.660
[Epoch 1 Batch 750/1407] loss=8.0632, lr=0.0000496210, acc=0.661
[Epoch 1 Batch 800/1407] l